In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchinfo import summary
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

In [ ]:
# 1. Configuration and Data Loading
BATCH_SIZE = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    # Normalize with MNIST mean (0.1307) and std (0.3081)
    transforms.Normalize((0.1307,), (0.3081,))
])

In [ ]:
# Download and load training data
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Download and load test data
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [ ]:
# Use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# 2. Define the Neural Network Model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Input layer (28*28 = 784 pixels) to a hidden layer of 128 neurons
        self.fc1 = nn.Linear(28 * 28, 128)
        # Second hidden layer
        self.fc2 = nn.Linear(128, 64)
        # Output layer (10 classes for digits 0-9)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        # Flatten the input image from 28x28 to a 784-element vector
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # Apply log softmax for the final output (often used with NLLLoss, which is part of CrossEntropyLoss)
        return F.softmax(x, dim=1)


model = SimpleNN().to(device)

In [ ]:
# 3. Define Loss and Optimizer
criterion = nn.NLLLoss()  # Negative Log-Likelihood Loss (suitable for log_softmax output)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
summary(model)

In [ ]:
writer = SummaryWriter('runs/init')

In [ ]:
# 4. Training Loop
NUM_EPOCHS = 5

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()  # Clear gradients
        output = model(data)  # Forward pass
        loss = criterion(output, target)  # Calculate loss
        loss.backward()  # Backward pass (calculate gradients)
        optimizer.step()  # Update weights

        running_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch + 1}/{NUM_EPOCHS} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    avg_train_loss = running_loss / len(train_loader)
    writer.add_scalar('Loss/Train', avg_train_loss, epoch)

writer.flush()
writer.close()

In [ ]:
model.eval()

In [ ]:
model.cpu()

In [ ]:
# tensorboard --logdir=runs

In [ ]:
# 5. Testing the Model
test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)

print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
      f'({accuracy:.2f}%)\n')

In [ ]:
# Save the model (optional)
torch.save(model.state_dict(), "mnist_simple_nn.pth")

In [ ]:
def export_onnx(model, onnx_path="dynamic_cnn.onnx"):
    dummy = torch.randn(1, 3, 224, 224)

    with torch.inference_mode():
        torch.onnx.export(
            model,
            dummy,
            onnx_path,
            opset_version=18,
            dynamo=False,
            export_params=True,
            do_constant_folding=True,
            input_names=["input"],
            output_names=["output"],
            dynamic_axes={
                "input": {0: "batch", 2: "height", 3: "width"},
                "output": {0: "batch"},
            },
        )

In [ ]:
onnx_path = "model.onnx"
export_onnx(model, onnx_path)

In [ ]:
import onnx

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)  # Check for model correctness

print("Model inputs:")
for input in onnx_model.graph.input:
    print(f"Input name: {input.name}")
    print(
        f"Input shape: {[(d.dim_value if d.HasField('dim_value') else d.dim_param) for d in input.type.tensor_type.shape.dim]}")

In [ ]:
# Take a look at model graph
# https://netron.app/